In [54]:
using PyPlot;
using LinearAlgebra, Statistics, HDF5, Images

train_data = read(h5open("/pnas/liujiang_group/yinyao/codes/repos/DataLists/train_catvnoncat.h5"))
test_data = read(h5open("/pnas/liujiang_group/yinyao/codes/repos/DataLists/test_catvnoncat.h5"))

train_X = Int.(train_data["train_set_x"]) ./ 255
train_Y = train_data["train_set_y"]
test_X = Int.(test_data["test_set_x"]) ./ 255
test_Y = test_data["test_set_y"]
;

In [66]:
function sigmoid(x)
    1 / (1 + exp(-x))
end

function zero_init(dim)
    w = zeros(dim)
    b = 0
    w, b
end

function propagate(w, b, X, Y)
    m = size(X)[2]
    Z = w' * X .+ b
    A = sigmoid.(Z)
    cost = -mean(Y .* log.(A) .+ (1 .- Y) .* log.(1 .- A))
    dw = X * (A - Y)' ./ m
    db = mean(A - Y)
    dw, db, cost
end

function optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = false)
    for i in 1:num_iterations
        dw, db, cost = propagate(w, b, X, Y)
        w -= learning_rate .* dw
        b -= learning_rate * db
        if print_cost && i % 100 == 0
            println("Cost after iteration $(i): $(round(cost, digits = 4))")
        end
    end
    w, b
end

function predict(w, b, X)
    m = size(X)[2]
    A = sigmoid.(w' * X .+ b)
    Y_pred = ifelse.(A .<= 0.5, 0, 1)
end

function model(train_X, train_Y, test_X, test_Y; num_iterations = 2000, learning_rate = 0.009, print_cost = false)
    train_X = reshape(train_X, :, size(train_X)[length(size(train_X))])
    train_Y = reshape(train_Y, :, size(train_Y)[length(size(train_Y))])
    test_X = reshape(test_X, :, size(test_X)[length(size(test_X))])
    test_Y = reshape(test_Y, :, size(test_Y)[length(size(test_Y))])
    n = size(train_X)[1]    
    w, b = zero_init(n)
    w, b = optimize(w, b, train_X, train_Y, num_iterations, learning_rate, print_cost)
    train_Y_pred = predict(w, b, train_X)
    test_Y_pred = predict(w, b, test_X)
    
    println("train accuracy: $(round(mean(train_Y_pred .== train_Y) * 100, digits = 2))%")
    println("test accuracy: $(round(mean(test_Y_pred .== test_Y) * 100, digits = 2))%")
    w, b
end

model(train_X, train_Y, test_X, test_Y, num_iterations = 2000, print_cost = true);

Cost after iteration 100: 0.4772
Cost after iteration 200: 0.8471
Cost after iteration 300: 0.5838
Cost after iteration 400: 0.4811
Cost after iteration 500: 0.3612
Cost after iteration 600: 0.2412
Cost after iteration 700: 0.1686
Cost after iteration 800: 0.1505
Cost after iteration 900: 0.1396
Cost after iteration 1000: 0.1304
Cost after iteration 1100: 0.1224
Cost after iteration 1200: 0.1153
Cost after iteration 1300: 0.109
Cost after iteration 1400: 0.1033
Cost after iteration 1500: 0.0982
Cost after iteration 1600: 0.0935
Cost after iteration 1700: 0.0893
Cost after iteration 1800: 0.0854
Cost after iteration 1900: 0.0818
Cost after iteration 2000: 0.0785
train accuracy: 99.52%
test accuracy: 70.0%


([0.0132083; -0.0357096; … ; -0.0378226; 0.0330869], -0.0060244826376061825)

In [ ]:
newimg = load("/pnas/liujiang_group/yinyao/codes/repos/DataLists/cat.jfif")


┌ Info: Precompiling ImageMagick [6218d12a-5da1-5696-b52f-db25d2ecc6d1]
└ @ Base loading.jl:1186
